In [12]:
import pandas as pd 
import time
import random
import os
from utils import get_data,data_processing
from datetime import datetime
import uuid
import ssl

In [13]:
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
url = ['https://www.espn.com.co/futbol/posiciones/_/liga/esp.1',
      'https://www.espn.com.co/futbol/posiciones/_/liga/eng.1',
      'https://www.espn.com.co/futbol/posiciones/_/liga/ita.1',
      'https://www.espn.com.co/futbol/posiciones/_/liga/ger.1',
      'https://www.espn.com.co/futbol/posiciones/_/liga/fra.1',
      'https://www.espn.com.co/futbol/posiciones/_/liga/por.1',
      'https://www.espn.com.co/futbol/posiciones/_/liga/ned.1']
ligas = ['ESPAÑA','INGLATERRA','ITALIA','GERMANY','FRANCIA','PORTUGAL','HOLANDA']

df_ligas = {
    'LIGA':ligas,
    'URL':url
} 

df_ligas = pd.DataFrame(df_ligas)
df_ligas

,LIGA,URL
0,ESPAÑA,https://www.espn.com.co/futbol/posiciones/_/li...
1,INGLATERRA,https://www.espn.com.co/futbol/posiciones/_/li...
2,ITALIA,https://www.espn.com.co/futbol/posiciones/_/li...
3,GERMANY,https://www.espn.com.co/futbol/posiciones/_/li...
4,FRANCIA,https://www.espn.com.co/futbol/posiciones/_/li...
5,PORTUGAL,https://www.espn.com.co/futbol/posiciones/_/li...
6,HOLANDA,https://www.espn.com.co/futbol/posiciones/_/li...


In [3]:
def get_data(url,liga):

    tiempo = [5,3,8]
    time.sleep(random.choice(tiempo))
    df = pd.read_html(url)
    df=pd.concat([df[0],df[1]],ignore_index=True,axis=1)
    df=df.rename(columns={0:'EQUIPO',1:'J', 2:'G', 3:'E', 4:'P', 5:'GF', 6:'GC', 7:'DIF', 8:'PTS'})
    df['EQUIPO']=df['EQUIPO'].apply(lambda x: x[5:] if x[:2].isnumeric()==True else x[4:])
    df['LIGA'] = liga

    run_date = datetime.now()
    run_date = run_date.strftime("%Y-%m-%d")
    df['CREATED_AT'] = run_date

    return df

def data_processing(df):

    df_spain=get_data(df['URL'][0],df['LIGA'][0])
    df_premier=get_data(df['URL'][1],df['LIGA'][1])
    df_italy=get_data(df['URL'][2],df['LIGA'][2])
    df_germany=get_data(df['URL'][3],df['LIGA'][3])
    df_francia=get_data(df['URL'][4],df['LIGA'][4])
    df_portugal=get_data(df['URL'][5],df['LIGA'][5])
    df_holanda=get_data(df['URL'][6],df['LIGA'][6])

    df_final=pd.concat([df_spain,df_premier,df_italy,df_francia,df_portugal,df_holanda],ignore_index=False)

    return df_final

In [8]:
df = pd.read_csv('/aws/DE-PRJCTS/SnowflakeAirflow/SnowflakeAirflow/AIRFLOW_DEPLOY/df_ligas.csv')
df_team = pd.read_csv('/aws/DE-PRJCTS/SnowflakeAirflow/SnowflakeAirflow/AIRFLOW_DEPLOY/team_table.csv')

In [17]:
df_data = data_processing(df)
df_final = pd.merge(df_data,df_team,how='inner',on='EQUIPO')
df_final = df_final[['ID_TEAM','EQUIPO', 'J', 'G', 'E', 'P', 'GF', 'GC', 'DIF', 'PTS', 'LIGA', 'CREATED_AT']]
df_final.to_csv('./premier_positions.csv',index=False)

In [14]:
df_ligas['URL'][1]

'https://www.espn.com.co/futbol/posiciones/_/liga/eng.1'

In [16]:
df2 = pd.read_html(df_ligas['URL'][1])
print(df2)

[                       2023/2024
0            1MCIManchester City
1          2TOTTottenham Hotspur
2                  3LIVLiverpool
3            4WHUWest Ham United
4     5BHABrighton & Hove Albion
5                    6ARSArsenal
6             7CRYCrystal Palace
7                  8BRNBrentford
8          9FORNottingham Forest
9         10MUNManchester United
10              11AVLAston Villa
11                  12CHEChelsea
12                   13FULFulham
13         14NEWNewcastle United
14  15WOLWolverhampton Wanderers
15          16BOUAFC Bournemouth
16   17Sheff UtdSheffield United
17                  18EVEEverton
18               19LTNLuton Town
19                  20BRNBurnley,     J  G  E  P  GF  GC  DIF  PTS
0   4  4  0  0  11   2    9   12
1   4  3  1  0  11   4    7   10
2   4  3  1  0   9   3    6   10
3   4  3  1  0   9   4    5   10
4   4  3  0  1  12   6    6    9
5   3  2  1  0   5   3    2    7
6   4  2  1  1   5   4    1    7
7   4  1  3  0   8   5    3    6
8   4  2

In [18]:
len(df)

7

In [ ]:
df[0]

In [8]:
df[1]

,J,G,E,P,GF,GC,DIF,PTS
0,7,6,0,1,17,7,10,18
1,7,5,2,0,23,6,17,17
2,7,5,2,0,18,7,11,17
3,6,4,1,1,11,5,6,13
4,6,4,0,2,8,8,0,12
5,7,3,2,2,12,11,1,11
6,6,3,1,2,8,9,-1,10
7,6,2,3,1,15,6,9,9
8,7,2,3,2,15,12,3,9
9,7,1,5,1,8,7,1,8


In [9]:
df=pd.concat([df[0],df[1]],ignore_index=True,axis=1)
df

,0,1,2,3,4,5,6,7,8
0,1ARSArsenal,7,6,0,1,17,7,10,18
1,2MCIManchester City,7,5,2,0,23,6,17,17
2,3TOTTottenham Hotspur,7,5,2,0,18,7,11,17
3,4BHABrighton & Hove Albion,6,4,1,1,11,5,6,13
4,5MUNManchester United,6,4,0,2,8,8,0,12
5,6FULFulham,7,3,2,2,12,11,1,11
6,7CHEChelsea,6,3,1,2,8,9,-1,10
7,8LIVLiverpool,6,2,3,1,15,6,9,9
8,9BRNBrentford,7,2,3,2,15,12,3,9
9,10NEWNewcastle United,7,1,5,1,8,7,1,8


In [10]:
df=df.rename(columns={0:'EQUIPO',1:'J', 2:'G', 3:'E', 4:'P', 5:'GF', 6:'GC', 7:'DIF', 8:'PTS'})
df['EQUIPO']=df['EQUIPO'].apply(lambda x: x[5:] if x[:2].isnumeric()==True else x[4:])
df['LIGA'] = df_ligas['LIGA'][1]

In [11]:
df

,EQUIPO,J,G,E,P,GF,GC,DIF,PTS,LIGA
0,Arsenal,7,6,0,1,17,7,10,18,INGLATERRA
1,Manchester City,7,5,2,0,23,6,17,17,INGLATERRA
2,Tottenham Hotspur,7,5,2,0,18,7,11,17,INGLATERRA
3,Brighton & Hove Albion,6,4,1,1,11,5,6,13,INGLATERRA
4,Manchester United,6,4,0,2,8,8,0,12,INGLATERRA
5,Fulham,7,3,2,2,12,11,1,11,INGLATERRA
6,Chelsea,6,3,1,2,8,9,-1,10,INGLATERRA
7,Liverpool,6,2,3,1,15,6,9,9,INGLATERRA
8,Brentford,7,2,3,2,15,12,3,9,INGLATERRA
9,Newcastle United,7,1,5,1,8,7,1,8,INGLATERRA
